Imran Muhammad, Olga Kellert, and Carlos Gómez-Rodríguez. "A Syntax-Injected Approach for Faster and More Accurate Sentiment Analysis." arXiv preprint arXiv:2406.15163 (2024). The preprint version is available on arXiv: URL: https://arxiv.org/html/2406.15163v1

### Install Libraries

In [1]:
!pip install matplotlib
!pip install datasets
!pip install easydict
!pip install frozendict
!pip install transformers
!pip install conllu
!pip install evaluate
!pip install seqeval
!pip install accelerate -U

### Import Libraries

In [2]:
import datasets
from datasets import Sequence, Value, ClassLabel, Features, load_dataset

from src.models.deps_tree import D_Tree
from src.models.deps_label import D_Label
from src.utils.constants import D_ROOT_HEAD, D_NULLHEAD, D_ROOT_REL, D_POSROOT, D_EMPTYREL, D_2P_GREED, D_2P_PROP
from src.encs.enc_deps import *
from src.utils.constants import *
from src.models.linearized_tree import LinearizedTree

import easydict
from frozendict import frozendict
import os
import torch
import pandas as pd
import numpy as np
import json
from tempfile import NamedTemporaryFile

from utilities import *
import transformers
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

import evaluate
import spacy

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Load Dataset

In [4]:
ud_path = "../datasets/UD_English-EWT/"
ud_path = os.path.expanduser(ud_path)

with open(os.path.join(ud_path,"en_ewt-ud-test.conllu")) as f:
    ud_test = [p.strip() for p in f.read().split('\n\n')]
with open(os.path.join(ud_path,"en_ewt-ud-dev.conllu")) as f:
    ud_dev = [p.strip() for p in f.read().split('\n\n')]
with open(os.path.join(ud_path,"en_ewt-ud-train.conllu")) as f:
    ud_train = [p.strip() for p in f.read().split('\n\n')]

In [5]:
print(ud_test[0])

# newdoc id = weblog-blogspot.com_zentelligence_20040423000200_ENG_20040423_000200
# sent_id = weblog-blogspot.com_zentelligence_20040423000200_ENG_20040423_000200-0001
# newpar id = weblog-blogspot.com_zentelligence_20040423000200_ENG_20040423_000200-p0001
# text = What if Google Morphed Into GoogleOS?
1	What	what	PRON	WP	PronType=Int	0	root	0:root	_
2	if	if	SCONJ	IN	_	4	mark	4:mark	_
3	Google	Google	PROPN	NNP	Number=Sing	4	nsubj	4:nsubj	_
4	Morphed	morph	VERB	VBD	Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin	1	advcl	1:advcl:if	_
5	Into	into	ADP	IN	_	6	case	6:case	_
6	GoogleOS	GoogleOS	PROPN	NNP	Number=Sing	4	obl	4:obl:into	SpaceAfter=No
7	?	?	PUNCT	.	_	4	punct	4:punct	_


### Decode UD Treebank using CoDeLin Library

In [6]:
def encode(conllu_samples):
    samples = []
    for conllu_sample in conllu_samples:
        d_tree = D_Tree.from_string(conllu_sample)
        encoder = D_NaiveRelativeEncoding(separator="_", hang_from_root=True)
        ld_tree = encoder.encode(d_tree)
        samples.append(remove_bos_eos_tags(ld_tree))
    return samples

def remove_bos_eos_tags(text):
    sentence = text.to_string()
    lines = sentence.split('\n')
    lines = lines[1:-2]
    sentence = '\n'.join(lines)
    return sentence

In [7]:
ud_test_original = ud_test
ud_test = encode(ud_test)
ud_dev = encode(ud_dev)
ud_train = encode(ud_train)

print(ud_test[0])

What	PRON	-NONE-_root
if	SCONJ	2_mark
Google	PROPN	1_nsubj
Morphed	VERB	-3_advcl
Into	ADP	1_case
GoogleOS	PROPN	-2_obl
?	PUNCT	-3_punct


In [8]:
def return_class_names(data_files):
    train_file = data_files["train"]
    eval_file = data_files["validation"]
    test_file = data_files["test"]

    class_names = {"pos_tags": set(), "syntax_labels": set()}

    for file in [train_file, eval_file, test_file]:
        for sentence in file:
            #sentence = sentence.to_string()
            sentence = str(sentence)
            lines = sentence.split('\n')
            for line in lines:
                if line:
                    token, pos_tag, syntax_label = line.split('\t')
                    class_names["pos_tags"].add(pos_tag)
                    if file != test_file:
                        class_names["syntax_labels"].add(syntax_label)

    class_names["pos_tags"] = list(class_names["pos_tags"])
    class_names["syntax_labels"] = list(class_names["syntax_labels"])

    # add UNK label
    class_names["syntax_labels"].append("UNK")

    return class_names

In [9]:
def create_json_files(data_files):
    """Convert the TSV data files in a JSON file to create a dataset"""
    train_json = NamedTemporaryFile("w", delete=False)
    eval_json = NamedTemporaryFile("w", delete=False)
    test_json = NamedTemporaryFile("w", delete=False) 

    train_list = []
    eval_list = []
    test_list = []
    
    class_names = return_class_names(data_files)

    for split in data_files:
        idx = 0
        file = data_files[split]
            
        for sentence in file:
            idx += 1
            sentence_tokens = []
            sentence_pos_tags = []
            sentence_syntax_labels = []
            sentence = str(sentence)
            lines = sentence.split('\n')
            
            if len(lines) > 1:
                for line in lines:
                    if line:
                        token, pos_tag, syntax_label = line.split('\t')
                        sentence_tokens.append(token)
                        sentence_pos_tags.append(pos_tag)
                        if split != "test":
                            sentence_syntax_labels.append(syntax_label)
                        else:
                            if syntax_label in class_names["syntax_labels"]:
                                sentence_syntax_labels.append(syntax_label)
                            else:
                                sentence_syntax_labels.append("UNK")

            if sentence_tokens != []:
                if split == "train":
                    train_list.append({"id": idx, "tokens": sentence_tokens, "pos_tags": sentence_pos_tags, "syntax_labels": sentence_syntax_labels})
                elif split == "validation":
                    eval_list.append({"id": idx, "tokens": sentence_tokens, "pos_tags": sentence_pos_tags, "syntax_labels": sentence_syntax_labels})
                else:
                    test_list.append({"id": idx, "tokens": sentence_tokens, "pos_tags": sentence_pos_tags, "syntax_labels": sentence_syntax_labels})

    train_dic = {"data": train_list}
    eval_dic = {"data": eval_list}
    test_dic = {"data": test_list}

    with open(train_json.name, 'w', encoding='utf-8') as f:
        json.dump(train_dic, f, ensure_ascii=False)

    with open(eval_json.name, 'w', encoding='utf-8') as f:
        json.dump(eval_dic, f, ensure_ascii=False)
    
    with open(test_json.name, 'w', encoding='utf-8') as f:
        json.dump(test_dic, f, ensure_ascii=False)

    return {"train":train_json.name, "validation":eval_json.name, "test":test_json.name}

In [10]:
seqeval = evaluate.load("seqeval")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [11]:
data_dir = 'data/const/single/bert-base-cased/english/'

data_tsv = {
    "train": ud_train, 
    "validation": ud_dev,
    "test": ud_test
}

data_json = create_json_files(data_tsv)
class_names = return_class_names(data_tsv)

num_labels = len(class_names["syntax_labels"])

label2id = {v: i for i, v in enumerate(class_names["syntax_labels"])}
id2label = {i: v for i, v in enumerate(class_names["syntax_labels"])}

### Initialize Model From Pretrained Checkpoint

In [12]:
# Initilize Pretrained Model
model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", # dccuchile/distilbert-base-spanish-uncased  for spanish datasets
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

# Initilize Tokenizer for Pretrained Model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", add_prefix_space=True)
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

dataset = load_dataset(
    "json",
    data_files=data_json,
    features=Features(
        {
            "id": Value("int64"),
            "tokens": Sequence(Value("string")),
            "pos_tags": Sequence(ClassLabel(names=class_names["pos_tags"], num_classes=len(class_names["pos_tags"]))),
            "syntax_labels": Sequence(ClassLabel(names=class_names["syntax_labels"], num_classes=len(class_names["syntax_labels"]))),
        }
    ),
    field="data"
)

syntax_features = dataset["train"].features["syntax_labels"]
label_names = syntax_features.feature.names

# Align Tokens and Labels
dataset = dataset.map(
    tokenize_and_align_labels, 
    batched=True,
    remove_columns=dataset["train"].column_names,
    fn_kwargs={"tokenizer": tokenizer})

# Save Tokenized Dataset to the Disk
dataset.save_to_disk('models/rel-distilbert-en/dataset')

features = dataset["train"].features["labels"]

# Model Hyperparameters
training_args = TrainingArguments(
    "models/rel-distilbert-en/model",
    per_device_train_batch_size=128,
    learning_rate=1e-4,
    num_train_epochs= 30,
    weight_decay=0.001,
    adam_epsilon= 1e-7,
    adam_beta1= 0.9,
    adam_beta2= 0.999,
    metric_for_best_model="f1",  # Choose "f1" as the metric to evaluate the best model
    greater_is_better=True, 
    evaluation_strategy = "epoch",
    save_strategy="epoch",
    #save_strategy = "no",
    load_best_model_at_end=True,
)

# Initilize Model Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

# Train the Model
trainer.train()
# Save Trained Model
trainer.save_model('models/rel-distilbert-en/model')

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

...

Saving the dataset (0/1 shards):   0%|          | 0/12023 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1897 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1926 [00:00<?, ? examples/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,2.657968,0.638423,0.624665,0.631469,0.513284
2,No log,1.631288,0.722078,0.713711,0.717870,0.649288
3,No log,1.302382,0.762317,0.755591,0.758939,0.704465
4,No log,1.147412,0.783457,0.779092,0.781268,0.735155
5,No log,1.046818,0.801235,0.796739,0.798981,0.756682
6,No log,1.002671,0.808286,0.804343,0.806310,0.766645
7,No log,0.974333,0.809351,0.805237,0.807289,0.771927
8,No log,0.948694,0.820910,0.817638,0.819271,0.783451
9,No log,0.930606,0.823969,0.820485,0.822223,0.788292
10,No log,0.923810,0.827766,0.824266,0.826013,0.794174


### Inference with Raw Text (not tokenized)

In [13]:
from transformers import AutoTokenizer

text = "What if Google Morphed Into GoogleOS?"
print(text)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
inputs = tokenizer(text, return_tensors="pt")
print(inputs)


input_srts = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
print(input_srts)

decoded_string = tokenizer.decode(inputs['input_ids'][0])
decoded_string = decoded_string.split()
decoded_string = " ".join(decoded_string[1:-1])
print(decoded_string)

model = AutoModelForTokenClassification.from_pretrained("models/rel-distilbert-en/model")
with torch.no_grad():
    logits = model(**inputs).logits
    
predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0][1:-1]]

print(predicted_token_class)

What if Google Morphed Into GoogleOS?
{'input_ids': tensor([[  101,  2054,  2065,  8224, 22822,  8458,  2098,  2046,  8224,  2891,
          1029,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
['[CLS]', 'what', 'if', 'google', 'mor', '##ph', '##ed', 'into', 'google', '##os', '?', '[SEP]']
what if google morphed into googleos?
['5_obj', '2_mark', '1_nsubj', '-NONE-_root', '-NONE-_root', '-NONE-_root', '1_case', '-2_obl', '-2_obl', '-3_punct']


### Inference with Pre-tokenized Text

In [14]:
text = "I don't eat very much pieces of chocolate."
tokens_with_spacy = get_tokens_by_spacy(text)
print(tokens_with_spacy)

tokenized_text = pre_tokenized_text(tokens_with_spacy)
# Example pre-tokenized text/data
#tokenized_text = ["[CLS]", 'I', 'do', "n't", 'eat', 'very', 'much', 'pieces', 'of', 'chocolate', '.', "[SEP]"]

# Convert tokens to input IDs using the tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
input_ids = tokenizer.convert_tokens_to_ids(tokenized_text)

# Create input tensors
input_ids = torch.tensor(input_ids).unsqueeze(0)  # Add a batch dimension

model = AutoModelForTokenClassification.from_pretrained("models/rel-distilbert-en/model")
with torch.no_grad():
    #logits = model(**inputs).logits
    logits = model(input_ids).logits
    
    predictions = torch.argmax(logits, dim=2)
    predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0][1:-1]]
    # Model's predictions/rules
    print(predicted_token_class)

['I', 'do', "n't", 'eat', 'very', 'much', 'pieces', 'of', 'chocolate', '.']
['3_nsubj', '2_aux', '1_punct', '-NONE-_root', '1_advmod', '1_amod', '-3_obj', '1_case', '-2_nmod', '-6_punct']


### Decoded Inference Trees or Predictions from the Model on Test Dataset

In [15]:
test_trees = ud_test_original
dec_trees = []
tree_cycles_ids = []
tree_cycles = []
with torch.no_grad():
    for idx, gold_tree in enumerate(test_trees):
        tree = D_Tree.from_string(gold_tree)
        
        words = tree.get_words()
        words = words[1:]
        postags = tree.get_postags()
        postags = postags[1:]
        sentence = " ".join(words)
        #sentence = sentence.replace('-ROOT-', '')
        sentence = sentence.strip()
        sentence = sentence.lower()
        print(sentence)
        
        tokenized_text = pre_tokenized_text(words)
        # Convert tokens to input IDs using the tokenizer
        tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
        input_ids = tokenizer.convert_tokens_to_ids(tokenized_text)

        # Create input tensors
        input_ids = torch.tensor(input_ids).unsqueeze(0)  # Add a batch dimension
        output = model(input_ids)
        #tokenized_input = tokenizer(sentence, return_tensors='pt')
        #output = model(**tokenized_input)
        logits = output.logits
        predictions = torch.argmax(logits, dim=2)
        true_predictions = [model.config.id2label[t.item()] for t in predictions[0][1:-1]]

        labels = []
        for p in true_predictions:
            if(p != '[CLS]' and p != '[SEP]'):
                labels.append(D_Label.from_string(p, sep="_"))
        #print(words)
        #print(postags)
        #print(labels)
        
        lin_tree = LinearizedTree(words=words, postags=postags,
                    additional_feats=[], labels=labels, n_feats=0)
        
        #encoder = D_PosBasedEncoding(separator="_")
        encoder = D_NaiveRelativeEncoding(separator="_", hang_from_root=True)
        #encoder = D_NaiveAbsoluteEncoding(separator="_")
        dec_tree = encoder.decode(lin_tree)
        dec_tree.postprocess_tree(search_root_strat=D_ROOT_HEAD, allow_multi_roots=False)
        dec_tree = str(dec_tree)
        dec_trees.append(dec_tree)
        print(dec_tree)
        
        # Localize sentences having "UD Error: There is a cycle in a sentence"
        if "root" not in dec_tree:
            tree_cycles_ids.append(idx)
            tree_cycles.append(dec_tree)

what if google morphed into googleos ?
1	What	_	PRON	_	_	0	root	_	_
2	if	_	SCONJ	_	_	4	mark	_	_
3	Google	_	PROPN	_	_	1	root	_	_
4	Morphed	_	VERB	_	_	3	goeswith	_	_
5	Into	_	ADP	_	_	6	case	_	_
6	GoogleOS	_	PROPN	_	_	4	obl	_	_
7	?	_	PUNCT	_	_	2	punct	_	_


what if google expanded on its search - engine ( and now e-mail ) wares into a full - fledged operating system ?
1	What	_	PRON	_	_	0	root	_	_
2	if	_	SCONJ	_	_	4	mark	_	_
3	Google	_	PROPN	_	_	4	nsubj	_	_
4	expanded	_	VERB	_	_	1	advcl	_	_
5	on	_	ADP	_	_	9	case	_	_
6	its	_	PRON	_	_	9	nmod:poss	_	_
7	search	_	NOUN	_	_	9	compound	_	_
8	-	_	PUNCT	_	_	9	punct	_	_
9	engine	_	NOUN	_	_	4	obl	_	_
10	(	_	PUNCT	_	_	9	punct	_	_
11	and	_	CCONJ	_	_	13	cc	_	_
12	now	_	ADV	_	_	13	advmod	_	_
13	e-mail	_	NOUN	_	_	10	conj	_	_
14	)	_	PUNCT	_	_	9	punct	_	_
15	wares	_	NOUN	_	_	7	punct	_	_
16	into	_	ADP	_	_	21	case	_	_
17	a	_	DET	_	_	21	det	_	_
18	full	_	ADV	_	_	20	amod	_	_
19	-	_	PUNCT	_	_	20	punct	_	_
20	fledged	_	ADJ	_	_	22	amod	_	_
21	operating	_	NOUN	_	_	

1	You	_	PRON	_	_	2	nsubj	_	_
2	have	_	VERB	_	_	0	root	_	_
3	to	_	PART	_	_	4	mark	_	_
4	see	_	VERB	_	_	2	xcomp	_	_
5	these	_	DET	_	_	6	det	_	_
6	slides	_	NOUN	_	_	4	obj	_	_
7	....	_	PUNCT	_	_	6	compound	_	_
8	they	_	PRON	_	_	10	nsubj	_	_
9	are	_	AUX	_	_	10	cop	_	_
10	amazing	_	ADJ	_	_	4	parataxis	_	_
11	.	_	PUNCT	_	_	3	punct	_	_


this fallujah operation my turn out to be the most important operation done by the us military since the end of the war .
1	This	_	DET	_	_	3	det	_	_
2	Fallujah	_	PROPN	_	_	3	compound	_	_
3	operation	_	NOUN	_	_	5	nsubj	_	_
4	my	_	AUX	_	_	5	nmod:poss	_	_
5	turn	_	VERB	_	_	0	root	_	_
6	out	_	ADP	_	_	5	compound:prt	_	_
7	to	_	PART	_	_	12	mark	_	_
8	be	_	AUX	_	_	12	cop	_	_
9	the	_	DET	_	_	12	det	_	_
10	most	_	ADV	_	_	11	advmod	_	_
11	important	_	ADJ	_	_	12	amod	_	_
12	operation	_	NOUN	_	_	6	xcomp	_	_
13	done	_	VERB	_	_	12	acl	_	_
14	by	_	ADP	_	_	17	case	_	_
15	the	_	DET	_	_	17	det	_	_
16	US	_	PROPN	_	_	17	compound	_	_
17	Military	_	NOUN	_	_	13	obl	_	_
18	since	_	AD

1	They	_	PRON	_	_	5	nsubj	_	_
2	are	_	AUX	_	_	5	aux	_	_
3	certainly	_	ADV	_	_	5	advmod	_	_
4	being	_	AUX	_	_	5	cop	_	_
5	nasty	_	ADJ	_	_	0	root	_	_
6	to	_	ADP	_	_	11	case	_	_
7	the	_	DET	_	_	11	det	_	_
8	United	_	ADJ	_	_	9	amod	_	_
9	Nations	_	PROPN	_	_	11	compound	_	_
10	Security	_	PROPN	_	_	11	compound	_	_
11	Council	_	PROPN	_	_	5	obl	_	_
12	in	_	ADP	_	_	13	case	_	_
13	connection	_	NOUN	_	_	6	obl	_	_
14	with	_	ADP	_	_	17	case	_	_
15	the	_	DET	_	_	17	det	_	_
16	anti-proliferation	_	NOUN	_	_	17	compound	_	_
17	treaty	_	NOUN	_	_	13	nmod	_	_
18	.	_	PUNCT	_	_	6	punct	_	_


the president has also said he would like to see israel wiped off the map which he could n't even begin to try without nuclear weapons .
1	The	_	DET	_	_	2	det	_	_
2	President	_	PROPN	_	_	5	nsubj	_	_
3	has	_	AUX	_	_	5	aux	_	_
4	also	_	ADV	_	_	5	advmod	_	_
5	said	_	VERB	_	_	0	root	_	_
6	he	_	PRON	_	_	8	nsubj	_	_
7	would	_	AUX	_	_	8	aux	_	_
8	like	_	VERB	_	_	5	ccomp	_	_
9	to	_	PART	_	_	10	mark	_	_
10	see	_	VERB	_	_	8	xcomp

1	Let	_	VERB	_	_	0	root	_	_
2	aggressive	_	ADJ	_	_	6	amod	_	_
3	(	_	PUNCT	_	_	6	punct	_	_
4	American	_	ADJ	_	_	6	amod	_	_
5	)	_	PUNCT	_	_	6	punct	_	_
6	leaders	_	NOUN	_	_	1	nsubj	_	_
7	and	_	CCONJ	_	_	8	cc	_	_
8	soldiers	_	NOUN	_	_	6	conj	_	_
9	know	_	VERB	_	_	4	xcomp	_	_
10	that	_	SCONJ	_	_	13	mark	_	_
11	we	_	PRON	_	_	13	nsubj	_	_
12	are	_	AUX	_	_	13	cop	_	_
13	capable	_	ADJ	_	_	9	ccomp	_	_
14	of	_	SCONJ	_	_	15	mark	_	_
15	protecting	_	VERB	_	_	13	advcl	_	_
16	the	_	DET	_	_	17	det	_	_
17	city	_	NOUN	_	_	15	obj	_	_
18	's	_	PART	_	_	17	compound	_	_
19	security	_	NOUN	_	_	15	obj	_	_
20	and	_	CCONJ	_	_	21	cc	_	_
21	safety	_	NOUN	_	_	19	conj	_	_
22	,	_	PUNCT	_	_	24	punct	_	_
23	and	_	CCONJ	_	_	24	cc	_	_
24	ask	_	VERB	_	_	13	conj	_	_
25	them	_	PRON	_	_	24	iobj	_	_
26	to	_	PART	_	_	27	mark	_	_
27	lift	_	VERB	_	_	24	xcomp	_	_
28	their	_	PRON	_	_	29	nmod:poss	_	_
29	hands	_	NOUN	_	_	27	obj	_	_
30	from	_	ADP	_	_	32	case	_	_
31	the	_	DET	_	_	32	det	_	_
32	city	_	NOUN	_	_	27	obl	_	_
33	.	_	PUNCT

1	BREYER	_	NOUN	_	_	2	nsubj	_	_
2	filed	_	VERB	_	_	0	root	_	_
3	a	_	DET	_	_	5	det	_	_
4	concurring	_	VERB	_	_	5	compound	_	_
5	opinion	_	NOUN	_	_	2	obj	_	_
6	,	_	PUNCT	_	_	2	punct	_	_
7	in	_	ADP	_	_	2	case	_	_
8	which	_	PRON	_	_	13	obl	_	_
9	KENNEDY	_	PROPN	_	_	15	nsubj	_	_
10	,	_	PUNCT	_	_	11	punct	_	_
11	SOUTER	_	PROPN	_	_	9	conj	_	_
12	,	_	PUNCT	_	_	14	punct	_	_
13	and	_	CCONJ	_	_	14	cc	_	_
14	GINSBURG	_	PROPN	_	_	9	conj	_	_
15	joined	_	VERB	_	_	7	acl:relcl	_	_
16	.	_	PUNCT	_	_	3	punct	_	_


kennedy filed an opinion concurring in part , in which souter , ginsburg , and breyer joined as to parts i and ii .
1	KENNEDY	_	PROPN	_	_	2	nsubj	_	_
2	filed	_	VERB	_	_	0	root	_	_
3	an	_	DET	_	_	4	det	_	_
4	opinion	_	NOUN	_	_	2	obj	_	_
5	concurring	_	VERB	_	_	4	appos	_	_
6	in	_	ADP	_	_	7	case	_	_
7	part	_	NOUN	_	_	2	obl	_	_
8	,	_	PUNCT	_	_	4	punct	_	_
9	in	_	ADP	_	_	10	case	_	_
10	which	_	PRON	_	_	16	obl	_	_
11	SOUTER	_	PROPN	_	_	17	nsubj	_	_
12	,	_	PUNCT	_	_	13	punct	_	_
13	GINSBURG	_	PROPN	_	_

...

1	The	_	DET	_	_	2	det	_	_
2	price	_	NOUN	_	_	6	nsubj	_	_
3	they	_	PRON	_	_	4	nsubj	_	_
4	gave	_	VERB	_	_	2	acl:relcl	_	_
5	was	_	AUX	_	_	6	cop	_	_
6	good	_	ADJ	_	_	0	root	_	_
7	so	_	ADV	_	_	9	advmod	_	_
8	I	_	PRON	_	_	9	nsubj	_	_
9	said	_	VERB	_	_	6	parataxis	_	_
10	hey	_	INTJ	_	_	12	discourse	_	_
11	this	_	PRON	_	_	12	nsubj	_	_
12	seems	_	VERB	_	_	9	ccomp	_	_
13	great	_	ADJ	_	_	12	xcomp	_	_
14	.	_	PUNCT	_	_	6	punct	_	_


after they showed up there was a little trouble to get my car unlocked , it took quite a bit of time but the job was well done .
1	After	_	SCONJ	_	_	3	mark	_	_
2	they	_	PRON	_	_	3	nsubj	_	_
3	showed	_	VERB	_	_	6	advcl	_	_
4	up	_	ADP	_	_	3	compound:prt	_	_
5	there	_	PRON	_	_	6	expl	_	_
6	was	_	AUX	_	_	0	root	_	_
7	a	_	DET	_	_	9	det	_	_
8	little	_	ADJ	_	_	9	amod	_	_
9	trouble	_	NOUN	_	_	6	nsubj	_	_
10	to	_	PART	_	_	11	mark	_	_
11	get	_	VERB	_	_	9	acl	_	_
12	my	_	PRON	_	_	13	nmod:poss	_	_
13	car	_	NOUN	_	_	11	obj	_	_
14	unlocked	_	VERB	_	_	11	xcomp	_	_
15	,	_	PUNCT	_	_	1

1	Clean	_	ADJ	_	_	0	root	_	_
2	and	_	CCONJ	_	_	3	cc	_	_
3	superb	_	ADJ	_	_	1	conj	_	_
4	.	_	PUNCT	_	_	1	punct	_	_


thanks for the great care !!!!
1	Thanks	_	NOUN	_	_	0	root	_	_
2	for	_	ADP	_	_	5	case	_	_
3	the	_	DET	_	_	5	det	_	_
4	great	_	ADJ	_	_	5	amod	_	_
5	care	_	NOUN	_	_	1	nmod	_	_
6	!!!!	_	PUNCT	_	_	1	punct	_	_


will definitely go back when i need medical care .
1	Will	_	AUX	_	_	3	aux	_	_
2	definitely	_	ADV	_	_	3	advmod	_	_
3	go	_	VERB	_	_	0	root	_	_
4	back	_	ADV	_	_	3	advmod	_	_
5	when	_	ADV	_	_	7	advmod	_	_
6	I	_	PRON	_	_	7	nsubj	_	_
7	need	_	VERB	_	_	3	advcl	_	_
8	medical	_	ADJ	_	_	9	amod	_	_
9	care	_	NOUN	_	_	7	obj	_	_
10	.	_	PUNCT	_	_	3	punct	_	_


excellence in glasgow
1	excellence	_	NOUN	_	_	0	root	_	_
2	in	_	ADP	_	_	3	case	_	_
3	glasgow	_	PROPN	_	_	1	nmod	_	_


i v just had my bmw z3 rear window replaced by the guys at kelvin trimmers .
1	I	_	PRON	_	_	4	nsubj	_	_
2	v	_	AUX	_	_	4	aux	_	_
3	just	_	ADV	_	_	4	advmod	_	_
4	had	_	VERB	_	_	0	root	_	_
5	my	_	PRON	_	_	9	nmod:pos

1	Believe	_	VERB	_	_	0	root	_	_
2	it	_	PRON	_	_	1	obj	_	_
3	or	_	CCONJ	_	_	4	cc	_	_
4	not	_	PART	_	_	1	conj	_	_
5	,	_	PUNCT	_	_	13	punct	_	_
6	but	_	CCONJ	_	_	13	cc	_	_
7	the	_	DET	_	_	9	det	_	_
8	channel	_	NOUN	_	_	9	compound	_	_
9	guide	_	NOUN	_	_	13	nsubj	_	_
10	has	_	AUX	_	_	13	aux	_	_
11	been	_	AUX	_	_	13	cop	_	_
12	most	_	ADV	_	_	13	advmod	_	_
13	helpful	_	ADJ	_	_	1	conj	_	_
14	to	_	ADP	_	_	17	case	_	_
15	my	_	PRON	_	_	17	nmod:poss	_	_
16	family	_	NOUN	_	_	17	compound	_	_
17	members	_	NOUN	_	_	13	obl	_	_
18	that	_	PRON	_	_	19	nsubj	_	_
19	visit	_	VERB	_	_	17	acl:relcl	_	_
20	and	_	CCONJ	_	_	23	cc	_	_
21	do	_	AUX	_	_	23	aux	_	_
22	n't	_	PART	_	_	23	nsubj	_	_
23	know	_	VERB	_	_	15	conj	_	_
24	where	_	ADV	_	_	26	advmod	_	_
25	to	_	PART	_	_	26	mark	_	_
26	start	_	VERB	_	_	23	ccomp	_	_
27	when	_	ADV	_	_	29	advmod	_	_
28	it	_	PRON	_	_	29	nsubj	_	_
29	comes	_	VERB	_	_	26	advcl	_	_
30	to	_	SCONJ	_	_	31	mark	_	_
31	watching	_	VERB	_	_	29	advcl	_	_
32	satellite	_	NOUN	_	_	33	compound	_	_
3

1	While	_	SCONJ	_	_	5	mark	_	_
2	other	_	ADJ	_	_	5	nsubj	_	_
3	may	_	AUX	_	_	5	aux	_	_
4	be	_	AUX	_	_	5	cop	_	_
5	ok	_	ADJ	_	_	18	advcl	_	_
6	waiting	_	VERB	_	_	5	advcl	_	_
7	in	_	ADP	_	_	8	case	_	_
8	line	_	NOUN	_	_	6	obl	_	_
9	at	_	ADP	_	_	10	case	_	_
10	Town	_	PROPN	_	_	6	obl	_	_
11	Bagel	_	PROPN	_	_	5	punct	_	_
12	we	_	PRON	_	_	14	nsubj	_	_
13	are	_	AUX	_	_	14	cop	_	_
14	happy	_	ADJ	_	_	0	root	_	_
15	with	_	ADP	_	_	17	case	_	_
16	the	_	DET	_	_	17	det	_	_
17	quality	_	NOUN	_	_	14	obl	_	_
18	and	_	CCONJ	_	_	19	cc	_	_
19	service	_	NOUN	_	_	17	conj	_	_
20	we	_	PRON	_	_	21	nsubj	_	_
21	get	_	VERB	_	_	19	acl:relcl	_	_
22	at	_	ADP	_	_	23	case	_	_
23	Family	_	PROPN	_	_	21	obl	_	_
24	Bagels	_	PROPN	_	_	5	punct	_	_


very mediocre donuts !
1	Very	_	ADV	_	_	2	advmod	_	_
2	Mediocre	_	ADJ	_	_	0	root	_	_
3	donuts	_	NOUN	_	_	2	discourse	_	_
4	!	_	PUNCT	_	_	2	punct	_	_


the donuts were very over proofed , making them stale and bready .
1	The	_	DET	_	_	2	det	_	_
2	Donuts	_	NOUN	_	_	5	nsubj	_	_
3	w

1	He	_	PRON	_	_	0	root	_	_
2	listens	_	VERB	_	_	1	root	_	_
3	and	_	CCONJ	_	_	5	cc	_	_
4	is	_	AUX	_	_	5	cop	_	_
5	excellent	_	ADJ	_	_	2	conj	_	_
6	in	_	SCONJ	_	_	7	case	_	_
7	diagnosing	_	VERB	_	_	5	obl	_	_
8	,	_	PUNCT	_	_	5	punct	_	_
9	addressing	_	VERB	_	_	5	advcl	_	_
10	and	_	CCONJ	_	_	11	cc	_	_
11	explaining	_	VERB	_	_	9	conj	_	_
12	the	_	DET	_	_	14	det	_	_
13	specific	_	ADJ	_	_	14	amod	_	_
14	issues	_	NOUN	_	_	10	obj	_	_
15	and	_	CCONJ	_	_	16	cc	_	_
16	suggesting	_	VERB	_	_	9	conj	_	_
17	exercises	_	NOUN	_	_	16	obj	_	_
18	to	_	PART	_	_	19	mark	_	_
19	use	_	VERB	_	_	17	acl	_	_
20	.	_	PUNCT	_	_	3	punct	_	_







In [16]:
print(dec_trees[0])

1	What	_	PRON	_	_	0	root	_	_
2	if	_	SCONJ	_	_	4	mark	_	_
3	Google	_	PROPN	_	_	1	root	_	_
4	Morphed	_	VERB	_	_	3	goeswith	_	_
5	Into	_	ADP	_	_	6	case	_	_
6	GoogleOS	_	PROPN	_	_	4	obl	_	_
7	?	_	PUNCT	_	_	2	punct	_	_




### REMOVE Sentences With Cycle 

In [17]:
# Remove trees with cycles 
dec_trees = [ele for idx, ele in enumerate(dec_trees) if idx not in tree_cycles_ids]
#print(len(dec_trees))

# print trees with cycles 
for item in tree_cycles:
    print(item)

1	that	_	PRON	_	_	2	det	_	_
2	's	_	AUX	_	_	6	parataxis	_	_
3	fine	_	ADJ	_	_	7	discourse	_	_
4	,	_	PUNCT	_	_	8	punct	_	_
5	i	_	PRON	_	_	8	nsubj	_	_
6	do	_	AUX	_	_	8	aux	_	_
7	n't	_	PART	_	_	8	advmod	_	_
8	want	_	VERB	_	_	1	parataxis	_	_
9	to	_	PART	_	_	10	mark	_	_
10	see	_	VERB	_	_	8	xcomp	_	_
11	you	_	PRON	_	_	10	obj	_	_
12	either	_	ADV	_	_	10	advmod	_	_
13	,	_	PUNCT	_	_	7	punct	_	_
14	i	_	PRON	_	_	16	nsubj	_	_
15	just	_	ADV	_	_	16	advmod	_	_
16	need	_	VERB	_	_	7	parataxis	_	_
17	to	_	PART	_	_	18	mark	_	_
18	make	_	VERB	_	_	16	xcomp	_	_
19	a	_	DET	_	_	20	det	_	_
20	cd	_	NOUN	_	_	18	obj	_	_
21	.	_	PUNCT	_	_	9	punct	_	_


1	i	_	PRON	_	_	4	nsubj	_	_
2	ca	_	AUX	_	_	1	aux	_	_
3	n't	_	PART	_	_	4	obl:npmod	_	_
4	wait	_	VERB	_	_	2	parataxis	_	_


1	?	_	PUNCT	_	_	0	punct	_	_


1	?	_	PUNCT	_	_	0	punct	_	_


1	Stacey	_	PROPN	_	_	2	nsubj	_	_
2	here	_	ADV	_	_	1	flat	_	_
3	is	_	AUX	_	_	2	cop	_	_
4	the	_	DET	_	_	6	det	_	_
5	Master	_	NOUN	_	_	6	compound	_	_
6	draft	_	NOUN	_	_	2	nsubj	_	_
7	we	_	PRON	_

### Decoded Gold Trees from the Test Dataset

In [18]:
gold_dec_trees = []
for gold_tree in test_trees:
    d_tree_gold = D_Tree.from_string(gold_tree)

    #encoder = D_PosBasedEncoding(separator="_")
    encoder = D_NaiveRelativeEncoding(separator="_", hang_from_root=True)
    #encoder = D_NaiveAbsoluteEncoding(separator="_")
    #print("\n[*] Linearized tree:")
    ld_tree_gold = encoder.encode(d_tree_gold)

    #print("\n[*] Decoded tree:")
    d_tree_gold = encoder.decode(ld_tree_gold)
    d_tree_gold.postprocess_tree(search_root_strat=D_ROOT_HEAD, allow_multi_roots=False)
    
    d_tree_gold = str(d_tree_gold)
    gold_dec_trees.append(d_tree_gold)
    print(d_tree_gold)

1	What	_	PRON	_	_	0	root	_	_
2	if	_	SCONJ	_	_	4	mark	_	_
3	Google	_	PROPN	_	_	4	nsubj	_	_
4	Morphed	_	VERB	_	_	1	advcl	_	_
5	Into	_	ADP	_	_	6	case	_	_
6	GoogleOS	_	PROPN	_	_	4	obl	_	_
7	?	_	PUNCT	_	_	4	punct	_	_


1	What	_	PRON	_	_	0	root	_	_
2	if	_	SCONJ	_	_	4	mark	_	_
3	Google	_	PROPN	_	_	4	nsubj	_	_
4	expanded	_	VERB	_	_	1	advcl	_	_
5	on	_	ADP	_	_	15	case	_	_
6	its	_	PRON	_	_	15	nmod:poss	_	_
7	search	_	NOUN	_	_	9	compound	_	_
8	-	_	PUNCT	_	_	9	punct	_	_
9	engine	_	NOUN	_	_	15	compound	_	_
10	(	_	PUNCT	_	_	9	punct	_	_
11	and	_	CCONJ	_	_	13	cc	_	_
12	now	_	ADV	_	_	13	advmod	_	_
13	e-mail	_	NOUN	_	_	9	conj	_	_
14	)	_	PUNCT	_	_	15	punct	_	_
15	wares	_	NOUN	_	_	4	obl	_	_
16	into	_	ADP	_	_	22	case	_	_
17	a	_	DET	_	_	22	det	_	_
18	full	_	ADV	_	_	20	advmod	_	_
19	-	_	PUNCT	_	_	20	punct	_	_
20	fledged	_	ADJ	_	_	22	amod	_	_
21	operating	_	NOUN	_	_	22	compound	_	_
22	system	_	NOUN	_	_	4	obl	_	_
23	?	_	PUNCT	_	_	4	punct	_	_


1	[	_	PUNCT	_	_	4	punct	_	_
2	via	_	ADP	_	_	4	case	_	_
3	Microsoft	_

1	This	_	DET	_	_	2	det	_	_
2	man	_	NOUN	_	_	4	nsubj	_	_
3	was	_	AUX	_	_	4	cop	_	_
4	polite	_	ADJ	_	_	0	root	_	_
5	,	_	PUNCT	_	_	6	punct	_	_
6	professional	_	ADJ	_	_	4	conj	_	_
7	,	_	PUNCT	_	_	8	punct	_	_
8	clean	_	ADJ	_	_	4	conj	_	_
9	and	_	CCONJ	_	_	10	cc	_	_
10	quick	_	ADJ	_	_	4	conj	_	_
11	.	_	PUNCT	_	_	4	punct	_	_


1	I	_	PRON	_	_	3	nsubj	_	_
2	never	_	ADV	_	_	3	advmod	_	_
3	felt	_	VERB	_	_	0	root	_	_
4	worried	_	ADJ	_	_	3	xcomp	_	_
5	and	_	CCONJ	_	_	6	cc	_	_
6	walked	_	VERB	_	_	3	conj	_	_
7	away	_	ADV	_	_	6	advmod	_	_
8	satisfied	_	ADJ	_	_	6	xcomp	_	_
9	.	_	PUNCT	_	_	3	punct	_	_


1	THE	_	DET	_	_	2	det	_	_
2	TEACHING	_	NOUN	_	_	4	nsubj	_	_
3	THERE	_	ADV	_	_	2	advmod	_	_
4	SUCKS	_	VERB	_	_	0	root	_	_
5	!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!	_	PUNCT	_	_	4	punct	_	_


1	ALL	_	DET	_	_	8	nsubj	_	_
2	OF	_	ADP	_	_	4	case	_	_
3	THE	_	DET	_	_	4	det	_	_
4	TEACHERS	_	NOUN	_	_	1	nmod	_	_
5	THERE	_	ADV	_	_	1	advmod	_	_
6	ARE	_	AUX	_	_	8	cop	_	_
7	SO	_	ADV	_	_	8	advmod	_	_
8	MEAN	_	ADJ	_	_	0	roo

In [19]:
# Remove trees (with cycles) to make it consistent with predicted trees
gold_dec_trees = [ele for idx, ele in enumerate(gold_dec_trees) if idx not in tree_cycles_ids]
#print(len(gold_dec_trees))

### Write Decoded Trees into Files (gold, predicted)

In [20]:
gold_file_path = "../datasets/UD_English-EWT/gold-tress-test-decoded-rel.conllu"
# Delete previous files to avoid appending records again
if os.path.isfile(gold_file_path):
    os.remove(gold_file_path)
for t in gold_dec_trees:
    write_to_file(t, gold_file_path)

pred_file_path = "../datasets/UD_English-EWT/model-predictions-test-decoded-rel.conllu"
# Delete previous files to avoid appending records again
if os.path.isfile(pred_file_path):
    os.remove(pred_file_path)
for t in dec_trees:
    write_to_file(t, pred_file_path)

### Evaluate Model Performance using conll18_ud_eval.py

In [21]:
import subprocess

# Define the paths to the gold-standard and system-generated CoNLL-U files
gold_file = "../datasets/UD_English-EWT/gold-tress-test-decoded-rel.conllu"
system_file = "../datasets/UD_English-EWT-dataset/model-predictions-test-decoded-rel.conllu"

# Run the conll18_ud_eval.py script using subprocess
command = ["python", "/eval/conll18_ud_eval.py", gold_file, system_file]
result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Print the evaluation results
print(result.stdout)

# Check for errors, if any
if result.returncode != 0:
    print("Error occurred:", result.stderr)

LAS F1 Score: 76.17
MLAS Score: 71.14
BLEX Score: 73.31

